https://medium.com/@pytorch_geometric/link-prediction-on-heterogeneous-graphs-with-pyg-6d5c29677c70

这里，我们尝试，不要初始特征，让初始特征变成embedding。

In [1]:
## 装了个miniconda，python版本 3.12
## pip install torch torchvision torchaudio
## pip install torch_geometric
## pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.0+cpu.html

In [2]:
import os.path as osp
import torch
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

In [3]:
import pandas as pd
import numpy as np

In [4]:
# import torch_scatter, torch_sparse, torch_cluster, torch_spline_conv, torch_geometric

In [5]:
# pip install torch==2.1.1 torch-scatter==2.1.2 torch-sparse==0.6.18 torch-cluster==1.6.3 torch-spline-conv==1.2.2 torch-geometric==2.4.0 --target=./

In [6]:
cid_idx_relation__feaVal = pd.read_csv("test.csv")
df_ori = pd.read_csv("df_ori.csv")

In [7]:
from_colname, to_colname = "investor_id", "company_id"
fea_list = cid_idx_relation__feaVal.columns[6:-1]
len(fea_list)

70

In [8]:
# df_ori = pd.read_csv("sample.csv").head(10000).drop(columns = ["report_year"])

# node_order = sorted(list(set(df_ori.credit_code.to_list() + df_ori.outcredit_code.to_list())))

# mapping = {
#     ci: idx for idx, ci in enumerate(node_order)
# }
# for col in df_ori:
#     df_ori[col] = df_ori[col].map(mapping)

# 从这里搬：

In [11]:
from torch_geometric.data import Data
from torch.autograd import Variable

In [31]:
data = Data(
    x = Variable(
        torch.from_numpy(
            cid_idx_relation__feaVal[fea_list].to_numpy()
        ), 
        requires_grad=True
    ),
    num_nodes = cid_idx_relation__feaVal.shape[0],
    edge_index=torch.tensor(
        df_ori.T.to_numpy(), 
        dtype = torch.long
    )
)

In [32]:
data.is_directed()

True

In [33]:
n_nodes = data.num_nodes
hidden_channels = 64

In [34]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,    
)
train_data, val_data, test_data = transform(data)

In [35]:
from torch_geometric.loader import LinkNeighborLoader
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[
        10, 
        5
    ],
    neg_sampling_ratio=2.0,
    batch_size=128,
    shuffle=True,
)

In [36]:
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
class GNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, 128)
        self.conv2 = SAGEConv(128, out_channels)
    def forward(self, x, edge_index) :
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Classifier(torch.nn.Module):
    def forward(self, x_from, x_to,):
        return (x_from * x_to).sum(dim=-1)

class Model(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # self.emb = torch.nn.Embedding(n_nodes, in_channels)
        self.gnn = GNN(in_channels, out_channels)
        self.classifier = Classifier()
    def forward(self, data):
        x_out = data.x
        pred = self.classifier(
            x_out[data.edge_label_index[0]], ## 边的起始点。
            x_out[data.edge_label_index[-1]] ## 边的终结点。
        )
        return pred
        
model = Model(in_channels=hidden_channels, out_channels=64)

In [37]:
# !pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cu111.html
# import torch_geometric
# torch_geometric.typing.WITH_TORCH_SPARSE

In [42]:
import tqdm
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # ("mps")
print(f"Device: '{device}'")
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 10):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):  
        sampled_data.to(device)
        optimizer.zero_grad()
        pred = model(sampled_data)
        ground_truth = sampled_data.edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")
    

Device: 'cpu'


100%|████████████████████████████████████████████| 9/9 [00:00<00:00, 289.70it/s]


Epoch: 001, Loss: nan


100%|████████████████████████████████████████████| 9/9 [00:00<00:00, 583.39it/s]


Epoch: 002, Loss: nan


...

Epoch: 007, Loss: nan


100%|████████████████████████████████████████████| 9/9 [00:00<00:00, 533.46it/s]


Epoch: 008, Loss: nan


100%|████████████████████████████████████████████| 9/9 [00:00<00:00, 636.77it/s]

Epoch: 009, Loss: nan


In [43]:
# !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric

In [44]:
test_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[10, 5],
    neg_sampling_ratio=2.0,
    batch_size=128,
    shuffle=True,
)

In [45]:
from sklearn.metrics import roc_auc_score
preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(test_loader):
    with torch.no_grad():
        sampled_data.to(device)
        preds.append(model(sampled_data))
        ground_truths.append(sampled_data.edge_label)
pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC: {auc:.4f}")

100%|██████████████████████████████████████████| 13/13 [00:00<00:00, 634.86it/s]


ValueError: Input contains NaN.

In [47]:
sampled_data

Data(x=[624, 70], edge_index=[2, 321], num_nodes=624, edge_label=[327], edge_label_index=[2, 327], n_id=[624], e_id=[321], input_id=[109])

In [49]:
model(sampled_data).shape

torch.Size([327])

In [20]:
model.emb

Embedding(10272, 64)

In [22]:
weights = model.emb.weight.cpu().detach().numpy()
df_rst = pd.DataFrame(weights, columns = [f"col_{i}" for i in range(weights.shape[1])])
df_rst["company_id"] = node_order
df_rst.to_csv("embedding.csv", index=False)